# ML

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
import numpy as np
from skimpy import skim
import skimpy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
pd.set_option('display.max_columns', None)


In [2]:
data = 'C://Users//eduar//Desktop//DEV//Football//MLS//database.csv'
database = pd.read_csv(data)
database_ml = round(database.groupby('Jogador').agg({'Jogador': 'count'}),2)
database_ml.rename(columns={'Jogador':'Nº de Jogos'}, inplace= True)
database_ml.sort_values(by='Nº de Jogos', ascending= False, inplace= True)
database_ml = database_ml[database_ml['Nº de Jogos']>6].index
database_ml = database[database['Jogador'].isin(database_ml)]

In [3]:
database_ml.tail()

,Jogador,Time,#,Nação,Pos.,Idade,Min.,Gols,Assis.,PB,PT,TC,CaG,CrtsA,CrtV,Contatos,Div,Crts,Bloqueios,xG,npxG,xAG,SCA,GCA,Cmp,Att,Cmp%,PrgP,Conduções,PrgC,Tent,Suc,Data
12827,Ashley Westwood,Charlotte FC,8,ENG,DM,34-170,90,0,0,0,0,0,0,0,0,50,0,1,1,0.0,0.0,0.0,2,0,34,45,"75,6",5,34,0,0,0,2024-09-18
12828,Jere Uronen,Charlotte FC,21,FIN,LB,30-067,90,0,0,0,0,0,0,0,0,61,1,1,1,0.0,0.0,0.0,0,0,47,54,"87,0",5,24,1,0,0,2024-09-18
12830,Adilson Malanda,Charlotte FC,29,FRA,CB,22-325,90,0,0,0,0,0,0,0,0,59,0,3,0,0.0,0.0,0.0,0,0,47,51,"92,2",6,37,0,0,0,2024-09-18
12831,Nathan Byrne,Charlotte FC,14,ENG,RB,32-105,90,0,0,0,0,0,0,0,0,63,0,1,1,0.0,0.0,0.1,2,0,46,58,"79,3",2,45,0,0,0,2024-09-18
12832,Kristijan Kahlina,Charlotte FC,1,CRO,GK,32-056,90,0,0,0,0,0,0,0,0,27,0,0,0,0.0,0.0,0.0,0,0,14,25,"56,0",0,15,0,0,0,2024-09-18


# Agrupado por Jogador

In [5]:
# Agrupar os dados por jogador e equipe, e calcular as somas e contagens necessárias
db_agrupado = database.groupby(['Jogador', 'Equipe']).agg(
    Total_Gols=('Gols', 'sum'),           # Soma de gols
    Total_Chutes=('TC', 'sum'),           # Soma dos chutes totais (presumo que TC significa 'Total Chutes')
    Aparicoes=('Jogador', 'count'),       # Contagem de aparições
    Total_Chutes_Gol=('CaG', 'sum')       # Soma dos chutes ao gol (CaG = Chutes ao Gol)
)

# Calcular as médias: chute total, chute ao gol e gols por aparição
db_agrupado['Média_Chutes_Total'] = round(db_agrupado['Total_Chutes'] / db_agrupado['Aparicoes'], 2)
db_agrupado['Média_Chutes_Gol'] = round(db_agrupado['Total_Chutes_Gol'] / db_agrupado['Aparicoes'], 2)
db_agrupado['Média_Gols'] = round(db_agrupado['Total_Gols'] / db_agrupado['Aparicoes'], 2)

# Ordenar o DataFrame pela média de chutes ao gol, de forma decrescente
db_agrupado.sort_values(by='Média_Chutes_Gol', ascending=False, inplace=True)

# Resetar o índice para que 'Jogador' e 'Equipe' voltem a ser colunas normais (se necessário)
db_agrupado.reset_index(inplace=True)

db_agrupado = db_agrupado[(db_agrupado['Aparicoes']>6) & (db_agrupado['Total_Gols'] > 2)]


In [6]:
db_agrupado = db_agrupado[(db_agrupado['Aparicoes']>6) & (db_agrupado['Total_Gols'] > 2)]

In [7]:
db_agrupado

,Jogador,Equipe,Total_Gols,Total_Chutes,Aparicoes,Total_Chutes_Gol,Média_Chutes_Total,Média_Chutes_Gol,Média_Gols
0,Lionel Messi,Inter Miami,12,57,12,25,4.75,2.08,1.00
3,Luis Suárez,Inter Miami,16,52,20,37,2.60,1.85,0.80
4,Gabriel Chaves,LA Galaxy,10,89,26,39,3.42,1.50,0.38
7,Cucho,Crew,13,99,19,28,5.21,1.47,0.68
8,Carlos Gómez,RSL,13,66,22,32,3.00,1.45,0.59
...,...,...,...,...,...,...,...,...,...
289,Rémi Walter,Sporting KC,3,6,14,3,0.43,0.21,0.21
325,Mohamed Farsi,Crew,3,10,22,4,0.45,0.18,0.14
342,Jake Davis,Sporting KC,3,14,23,4,0.61,0.17,0.13
350,Jon Gallagher,Austin,3,9,25,4,0.36,0.16,0.12


In [10]:
times = db_agrupado['Equipe'].isin(['Portland Timbers','Rapids'])
db_agrupado[times]

,Jogador,Equipe,Total_Gols,Total_Chutes,Aparicoes,Total_Chutes_Gol,Média_Chutes_Total,Média_Chutes_Gol,Média_Gols
14,Jonathan Rodríguez,Portland Timbers,13,68,22,29,3.09,1.32,0.59
20,Evander,Portland Timbers,12,64,22,26,2.91,1.18,0.55
38,Rafael Navarro,Rapids,12,59,26,26,2.27,1.00,0.46
45,Felipe Mora,Portland Timbers,13,47,22,20,2.14,0.91,0.59
71,Djordje Mihailovic,Rapids,10,37,21,16,1.76,0.76,0.48
74,Cole Bassett,Rapids,8,60,26,19,2.31,0.73,0.31
102,Santiago Moreno,Portland Timbers,6,46,25,15,1.84,0.60,0.24
109,Kevin Cabral,Rapids,5,30,21,12,1.43,0.57,0.24
157,Antony,Portland Timbers,4,28,26,11,1.08,0.42,0.15
177,Calvin Harris,Rapids,3,17,24,9,0.71,0.38,0.12


In [20]:
database[database['Jogador'].isin(['Rafael Navarro'])]

,Equipe,Jogador,#,Nação,Pos.,Idade,Min.,Gols,Assis.,PB,PT,TC,CaG,CrtsA,CrtV,Contatos,Div,Crts,Bloqueios,xG,npxG,xAG,SCA,GCA,Cmp,Att,Cmp%,PrgP,Conduções,PrgC,Tent,Suc,Data
144,Rapids,Rafael Navarro,9.0,BRA,FW,23-316,76,0,0,0,0,2,1,0,0,32,2,1,2,0.1,0.1,0.0,2,0,14,19,"73,7",3,15,0,4,2,2024-02-24
765,Rapids,Rafael Navarro,9.0,BRA,FW,23-323,71,0,0,0,0,1,0,0,0,21,1,0,1,0.1,0.1,0.0,0,0,8,10,"80,0",0,10,0,1,0,2024-03-02
1374,Rapids,Rafael Navarro,9.0,BRA,FW,23-337,90,0,0,0,0,2,1,0,0,34,2,0,2,0.1,0.1,0.0,2,0,12,20,"60,0",1,17,2,3,2,2024-03-16
1893,Rapids,Rafael Navarro,9.0,BRA,FW,23-344,69,0,0,0,0,1,0,0,0,26,2,0,1,0.0,0.0,0.0,1,0,5,14,"35,7",0,11,1,3,1,2024-03-23
2050,Rapids,Rafael Navarro,9.0,BRA,FW,23-351,90,0,0,0,0,0,0,1,0,37,3,0,1,0.0,0.0,0.0,2,2,16,23,"69,6",0,20,3,5,3,2024-03-30
2523,Rapids,Rafael Navarro,9.0,BRA,FW,23-358,84,1,0,1,1,0,0,1,0,38,0,0,4,0.8,0.0,0.0,0,0,19,27,"70,4",0,22,3,5,4,2024-04-06
3141,Rapids,Rafael Navarro,9.0,BRA,FW,23-365,84,2,1,1,1,2,1,0,0,46,4,0,1,1.1,0.3,0.2,5,2,20,27,"74,1",1,22,3,7,2,2024-04-13
3507,Rapids,Rafael Navarro,9.0,BRA,FW,24-006,79,0,0,0,0,2,1,0,0,39,0,1,1,0.3,0.3,0.3,4,0,22,26,"84,6",3,25,3,3,1,2024-04-20
3739,Rapids,Rafael Navarro,9.0,BRA,FW,24-013,90,0,0,0,0,3,1,0,0,37,0,1,1,0.2,0.2,0.0,1,0,15,25,"60,0",2,24,1,3,2,2024-04-27
4562,Rapids,Rafael Navarro,9.0,BRA,FW,24-021,89,1,0,0,0,2,1,0,0,50,3,2,3,0.1,0.1,0.1,3,1,22,33,"66,7",6,25,1,1,0,2024-05-05


# Agrupado por Equipe

In [9]:
agrupado_time = database.groupby(['Equipe']).agg(
    Total_Gols_Time = ('Gols','sum'),
    Total_Chutes_Time = ('TC','sum'),
    Total_Chutes_Gol_Time = ('CaG','sum'),
    Total_Jogos = ('Data','nunique')
)

agrupado_time['Média_Gols']= round(agrupado_time['Total_Gols_Time'] / agrupado_time['Total_Jogos'],2)
agrupado_time['Média_Chutes']= round(agrupado_time['Total_Chutes_Time'] / agrupado_time['Total_Jogos'],2)
agrupado_time['Média_Chutes_Gols']= round(agrupado_time['Total_Chutes_Gol_Time'] / agrupado_time['Total_Jogos'],2)

agrupado_time.sort_values(by='Média_Chutes_Gols', ascending= False)

,Total_Gols_Time,Total_Chutes_Time,Total_Chutes_Gol_Time,Total_Jogos,Média_Gols,Média_Chutes,Média_Chutes_Gols
Equipe,,,,,,,
LAFC,47,391,147,25,1.88,15.64,5.88
LA Galaxy,52,375,149,27,1.93,13.89,5.52
Inter Miami,61,317,144,27,2.26,11.74,5.33
FC Cincinnati,48,387,146,28,1.71,13.82,5.21
NY Red Bulls,42,411,143,28,1.50,14.68,5.11
RSL,50,399,139,28,1.79,14.25,4.96
Philadelphia,49,419,140,29,1.69,14.45,4.83
Atlanta Utd,35,399,130,27,1.30,14.78,4.81
Minnesota Utd,43,337,125,27,1.59,12.48,4.63


# Agrupado por Data

In [8]:
agrupado_data = database.groupby(['Equipe','Data']).agg(
    Total_Gols_Time = ('Gols','sum'),
    Total_Chutes_Time = ('TC','sum'),
    Total_Chutes_Gol_Time = ('CaG','sum'),
    Total_Jogos = ('Data','nunique')
)

agrupado_data['Média_Gols']= round(agrupado_data['Total_Gols_Time'] / agrupado_data['Total_Jogos'],2)
agrupado_data['Média_Chutes']= round(agrupado_data['Total_Chutes_Time'] / agrupado_data['Total_Jogos'],2)
agrupado_data['Média_Chutes_Gols']= round(agrupado_data['Total_Chutes_Gol_Time'] / agrupado_data['Total_Jogos'],2)

agrupado_data.sort_values(by='Média_Chutes_Gols', ascending= False)

agrupado_data.reset_index(inplace=True)

agrupado_data

,Equipe,Data,Total_Gols_Time,Total_Chutes_Time,Total_Chutes_Gol_Time,Total_Jogos,Média_Gols,Média_Chutes,Média_Chutes_Gols
0,Atlanta Utd,2024-02-24,0,8,2,1,0.0,8.0,2.0
1,Atlanta Utd,2024-03-09,4,19,7,1,4.0,19.0,7.0
2,Atlanta Utd,2024-03-17,2,13,7,1,2.0,13.0,7.0
3,Atlanta Utd,2024-03-23,0,8,3,1,0.0,8.0,3.0
4,Atlanta Utd,2024-03-31,3,21,7,1,3.0,21.0,7.0
...,...,...,...,...,...,...,...,...,...
814,Vancouver W'caps,2024-07-13,4,13,9,1,4.0,13.0,9.0
815,Vancouver W'caps,2024-07-17,1,14,4,1,1.0,14.0,4.0
816,Vancouver W'caps,2024-07-20,3,15,6,1,3.0,15.0,6.0
817,Vancouver W'caps,2024-08-31,1,18,4,1,1.0,18.0,4.0


In [12]:
agrupado_data[agrupado_data['Equipe'].isin(['NY Red Bulls'])]

,Equipe,Data,Total_Gols_Time,Total_Chutes_Time,Total_Chutes_Gol_Time,Total_Jogos,Média_Gols,Média_Chutes,Média_Chutes_Gols
427,NY Red Bulls,2024-02-25,0,18,5,1,0.0,18.0,5.0
428,NY Red Bulls,2024-03-02,2,12,5,1,2.0,12.0,5.0
429,NY Red Bulls,2024-03-09,2,17,8,1,2.0,17.0,8.0
430,NY Red Bulls,2024-03-16,0,3,2,1,0.0,3.0,2.0
431,NY Red Bulls,2024-03-23,4,12,6,1,4.0,12.0,6.0
432,NY Red Bulls,2024-03-30,1,6,1,1,1.0,6.0,1.0
433,NY Red Bulls,2024-04-06,2,10,4,1,2.0,10.0,4.0
434,NY Red Bulls,2024-04-13,0,19,5,1,0.0,19.0,5.0
435,NY Red Bulls,2024-04-20,1,8,2,1,1.0,8.0,2.0
436,NY Red Bulls,2024-04-27,1,16,9,1,1.0,16.0,9.0


In [11]:
database['Equipe'].unique()

array(['Inter Miami', 'FC Cincinnati', 'RSL', 'Portland Timbers',
       'NE Revolution', 'Crew', 'LAFC', 'Atlanta Utd', 'Seattle',
       'SJ Earthquakes', 'Charlotte', 'FC Dallas', 'Fire', 'CF Montréal',
       'D.C. United', 'Philadelphia', 'Rapids', 'St. Louis', 'Austin',
       'Orlando City', 'Minnesota Utd', 'Dynamo FC', 'Sporting KC',
       'NYCFC', 'Toronto FC', 'NY Red Bulls', 'Nashville', 'LA Galaxy',
       "Vancouver W'caps", nan], dtype=object)